In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from typing import Dict, Tuple, Optional


"""
Ant-foraging model, implemented on a discrete 2D grid. Based on:
Wilensky, U. (1997). NetLogo Ants model. 
http://ccl.northwestern.edu/netlogo/models/Ants. 
Center for Connected Learning and Computer-Based Modeling, 
Northwestern University, Evanston, IL.


High-level idea
---------------
- The world is a grid of "patches" (cells). Each patch stores:
    * chemical: pheromone concentration (float)
    * food: number of food units (int, typically 0/1/2)
    * nest_mask: boolean "is this patch part of the nest?"
    * nest_scent: a static gradient that increases toward the nest center

- Ants are agents that live on the grid. Each ant stores:
    * (x, y): integer grid coordinates (column, row)
    * heading: orientation in degrees (0 = +x direction)
    * carrying: whether it currently carries food
    * who: unique ID used for "delayed departure" behavior

Per tick (world step)
---------------------
For each ant:
1) If it hasn't "started" yet (who >= tick), do nothing.
2) If searching (not carrying):
      - If standing on food: pick up 1 unit, turn around (180°), switch to returning.
      - Else: if pheromone at current patch is in the "chemical window":
              turn (±45°) toward higher pheromone (local hill-climb).
3) If returning (carrying):
      - If standing inside nest: drop food, turn around, switch to searching.
      - Else: deposit pheromone and turn toward higher nest_scent (home gradient).
4) Add random wiggle to heading (noise), then move forward one grid cell.

After all ants:
- Diffuse pheromone over the grid (8-neighborhood diffusion, non-wrapping edges)
- Evaporate pheromone (global decay)

Rendering
---------
We render:
- food as a green image
- pheromone as a blue overlay
- ants as points (orange if carrying food)
- the nest center as a red "X"
"""


# ============================================================
# World initialization
# ============================================================

def init_world(rng: np.random.Generator, max_pxcor: int = 50, max_pycor: int = 50, 
               food_radius: int = 5) -> Dict:
    """
    Create and initialize the patch grid and all patch-level fields.

    Parameters
    ----------
    rng : np.random.Generator
        Random generator (for reproducible food initialization).

    max_pxcor, max_pycor : int
            x in [-max_pxcor, +max_pxcor]
            y in [-max_pycor, +max_pycor]
        Internally we store arrays using index coordinates:
            x_index in [0, W-1], y_index in [0, H-1]
        where W = 2*max_pxcor+1 and H = 2*max_pycor+1.

    Returns
    -------
    world : dict
        A dictionary containing world dimensions and patch fields.
    """
    # Grid size in array indices
    H = 2 * max_pycor + 1
    W = 2 * max_pxcor + 1

    # Create arrays of (y_index, x_index) for every patch in the grid.
    # ys and xs are integer index coordinates (NOT world coordinates).
    ys, xs = np.mgrid[0:H, 0:W]

    # Convert indices to "world coordinates" centered at (0,0).
    # Here Xw,Yw are float arrays giving each patch's world-coordinate location.
    Xw = xs - max_pxcor
    Yw = ys - max_pycor

    # Patch fields (environment state)
    chemical = np.zeros((H, W), dtype=np.float64)      # pheromone concentration
    food = np.zeros((H, W), dtype=np.int32)            # food units on patch
    food_source_number = np.zeros((H, W), dtype=np.int32)  # which source (1/2/3), else 0

    # -----------------------------
    # Nest definition
    # -----------------------------
    # Nest is a disk of radius ~5 around the center (0,0).
    nest_radius = 5.0
    nest_mask = (np.sqrt(Xw**2 + Yw**2) < nest_radius)

    # "nest_scent" is a static field that points home.
    # Higher near the nest, lower far away. Returning ants follow this uphill.
    nest_scent = 200.0 - np.sqrt(Xw**2 + Yw**2)

    # -----------------------------
    # Food sources
    # -----------------------------
    # Food sources are 3 disks 
    # (0.6*max_pxcor, 0), (-0.6*max_pxcor,-0.6*max_pycor), (-0.8*max_pxcor, 
    # 0.8*max_pycor)
    sources = [
        (0.6 * max_pxcor, 0.0),
        (-0.6 * max_pxcor, -0.6 * max_pycor),
        (-0.8 * max_pxcor,  0.8 * max_pycor),
    ]

    for src_id, (sx, sy) in enumerate(sources, start=1):
        # Distance from every patch to the source center in world coords
        d = np.sqrt((Xw - sx)**2 + (Yw - sy)**2)
        mask = d < food_radius

        # Label which source this patch belongs to (1,2,3)
        food_source_number[mask] = src_id

        # Initialize food at source patches to either 1 or 2 randomly 
        # (like one-of [1 2])
        food[mask] = rng.choice([1, 2], size=int(mask.sum()))

    world = {
        "max_pxcor": max_pxcor,
        "max_pycor": max_pycor,
        "H": H,
        "W": W,
        "chemical": chemical,
        "food": food,
        "food_source_number": food_source_number,
        "nest_mask": nest_mask,
        "nest_scent": nest_scent,
    }
    return world


# ============================================================
# Ant initialization
# ============================================================

def init_ants(rng: np.random.Generator, world: Dict, population: int = 125) -> Dict:
    """
    Create a population of ants and place them initially inside the nest.

    Parameters
    ----------
    rng : np.random.Generator
        Random generator (for random initial headings and nest placement).

    world : dict
        The world dict produced by init_world(); used here to locate nest patches.

    population : int
        Number of ants.

    Returns
    -------
    ants : dict
        Ant state arrays:
          ants["x"], ants["y"] : integer grid indices
          ants["heading"]      : float degrees
          ants["carrying"]     : bool (food state)
          ants["who"]          : int unique IDs (0..population-1)
    """
    nest_indices = np.argwhere(world["nest_mask"])  # list of (y, x) index pairs 
                                                    # inside nest

    # Choose initial positions by sampling nest cells with replacement
    chosen = nest_indices[rng.integers(0, len(nest_indices), size=population)]
    ant_y = chosen[:, 0].astype(np.int32)
    ant_x = chosen[:, 1].astype(np.int32)

    ants = {
        "x": ant_x,
        "y": ant_y,
        # Headings are continuous angles but movement snaps to grid due to 
        # rounding
        "heading": rng.uniform(0, 360, size=population),
        # carrying=False means "search mode"; True means "return mode"
        "carrying": np.zeros(population, dtype=bool),
        # "who" implements delayed departure: ant i starts moving at tick >= i
        "who": np.arange(population, dtype=np.int32),
    }
    return ants


# ============================================================
# Movement + sensing helpers
# ============================================================

def in_bounds(nx: int, ny: int, W: int, H: int) -> bool:
    """
    Check whether (nx, ny) is inside the grid.

    (x, y) are array indices:
      x -> column index [0..W-1]
      y -> row index    [0..H-1]
    """
    return (0 <= nx < W) and (0 <= ny < H)


def patch_right_and_ahead(
    x: int, y: int,
    heading_deg: float,
    angle_deg: float,
    dist: int,
    W: int, H: int
) -> Optional[Tuple[int, int]]:
    """
    Given an ant at (x,y) with heading `heading_deg`, look at the patch that is:
      - rotated by `angle_deg` relative to the heading
      - `dist` steps away (in patch units)

    We turn the angle into a continuous direction vector, then snap it to the 
    grid by rounding dx, dy to the nearest integer step.

    Returns None if that target patch is outside the world.
    """
    theta = np.deg2rad(heading_deg + angle_deg)

    # Convert direction to a grid step. For dist=1, dx/dy become -1/0/+1.
    dx = int(np.rint(np.cos(theta) * dist))
    dy = int(np.rint(np.sin(theta) * dist))

    nx, ny = x + dx, y + dy
    if not in_bounds(nx, ny, W, H):
        return None
    return nx, ny


def scent_at_angle(
    field: np.ndarray,
    x: int, y: int,
    heading_deg: float,
    angle_deg: float,
    W: int, H: int
) -> float:
    """
    Sample a patch field one step away at a given relative angle.

    Example usage:
      - chemical ahead/right/left
      - nest_scent ahead/right/left

    If the sampled location is outside the world, return 0.0 (no scent).
    """
    p = patch_right_and_ahead(x, y, heading_deg, angle_deg, dist=1, W=W, H=H)
    if p is None:
        return 0.0
    nx, ny = p
    return float(field[ny, nx])  # note indexing order: [row=y, col=x]


def uphill(field: np.ndarray, ants: Dict, i: int, W: int, H: int) -> None:
    """
    Locally "climb" the scalar field by steering the ant left or right by 45 
    degrees.

    This implements the logic:
      sniff ahead, right, left
      if right or left is better than ahead:
          turn toward the better of right vs left (±45 degrees)

    Importantly:
    - This does NOT move the ant.
    - This only updates ants["heading"][i].
    - The actual move happens later via wiggle + forward.

    Effects on behavior
    -------------------
    - If field is nest_scent: returning ants gradually move toward the nest 
      center.
    - If field is chemical: searching ants can lock onto pheromone trails.
    """
    x, y, hdg = ants["x"][i], ants["y"][i], ants["heading"][i]

    ahead = scent_at_angle(field, x, y, hdg,   0, W, H)
    right = scent_at_angle(field, x, y, hdg,  45, W, H)
    left  = scent_at_angle(field, x, y, hdg, -45, W, H)

    # Only turn if a side direction beats straight-ahead.
    if (right > ahead) or (left > ahead):
        if right > left:
            ants["heading"][i] = (ants["heading"][i] + 45) % 360
        else:
            ants["heading"][i] = (ants["heading"][i] - 45) % 360


def wiggle(
    rng: np.random.Generator,
    ants: Dict,
    i: int,
    wiggle_max: int,
    W: int, H: int
) -> None:
    """
    Add random turning noise to the ant's heading and handle boundary collisions.

    Details
    -------
    - We add a random right turn and random left turn.
      The net turn is a difference of two uniforms, so it's centered around 0.
    - Then we check whether the ant *could* move forward 1 patch.
      If not, we flip heading by 180 degrees (bounce).

    Why wiggle exists
    -----------------
    Wiggle is the exploration engine:
    - prevents ants from moving deterministically forever
    - helps them discover trails/food by random wandering
    - spreads them out so pheromone trails can form
    """
    ants["heading"][i] = (ants["heading"][i] + rng.integers(0, wiggle_max)) % 360
    ants["heading"][i] = (ants["heading"][i] - rng.integers(0, wiggle_max)) % 360

    # Boundary check: if forward patch is outside the grid, bounce.
    p = patch_right_and_ahead(ants["x"][i], ants["y"][i], ants["heading"][i], 0, 
                              1, W, H)
    if p is None:
        ants["heading"][i] = (ants["heading"][i] + 180) % 360


def forward(ants: Dict, i: int, step_len: int, W: int, H: int) -> None:
    """
    Move the ant forward by step_len patches along its current heading.

    If forward movement would exit the grid, turn 180 degrees and do not move.
    """
    p = patch_right_and_ahead(ants["x"][i], ants["y"][i], ants["heading"][i], 0, 
                              step_len, W, H)
    if p is None:
        ants["heading"][i] = (ants["heading"][i] + 180) % 360
        return
    nx, ny = p
    ants["x"][i], ants["y"][i] = nx, ny


# ============================================================
# Chemical diffusion (8-neighborhood, non-wrapping)
# ============================================================

def diffuse_chemical(field: np.ndarray, rate_percent: float) -> np.ndarray:
    """
    Diffuse chemical over the grid using an 8-neighbor average (Moore 
    neighborhood).

    Parameters
    ----------
    field : np.ndarray
        Chemical concentration per patch.

    rate_percent : float
        Diffusion rate as a percentage.
        Conceptually: each tick, a fraction r of the chemical moves toward the
        local neighbor-average, smoothing the field.

    Returns
    -------
    np.ndarray
        New diffused field.

    Notes
    -----
    - We use NON-wrapping edges by padding with zeros.
      This mimics boundary behavior where outside-the-world patches are "empty".
    """
    r = rate_percent / 100.0
    if r <= 0:
        return field

    padded = np.pad(field, 1, mode="constant", constant_values=0.0)

    # center and 8 neighbors
    c  = padded[1:-1, 1:-1]
    n  = padded[0:-2, 1:-1]
    s  = padded[2:  , 1:-1]
    e  = padded[1:-1, 2:  ]
    w  = padded[1:-1, 0:-2]
    ne = padded[0:-2, 2:  ]
    nw = padded[0:-2, 0:-2]
    se = padded[2:  , 2:  ]
    sw = padded[2:  , 0:-2]

    neighbor_avg = (n + s + e + w + ne + nw + se + sw) / 8.0

    # Relaxation toward neighbor average:
    # - (1-r) keeps some of the old value
    # - r mixes in the neighbor mean
    return c * (1 - r) + neighbor_avg * r


# ============================================================
# One tick of the world
# ============================================================

def step_world(
    rng: np.random.Generator,
    ants: Dict,
    world: Dict,
    params: Dict,
    tick: int
) -> Dict[str, int]:
    """
    Perform one world tick: update all ants, then diffuse/evaporate chemical.

    Parameters
    ----------
    rng : np.random.Generator
        RNG used for wiggle noise.

    ants : dict
        Ant state arrays.

    world : dict
        Patch state arrays.

    params : dict
        Simulation parameters. Expected keys:
          - chem_follow_min, chem_follow_max : chemical "window" for following pheromone
          - deposit_amount                  : chemical deposited per step while returning
          - diffusion_rate                  : percent diffusion per tick
          - evaporation_rate                : percent evaporation per tick
          - wiggle_max                      : max random turn in wiggle
          - step_len                        : movement distance per tick (usually 1)

    tick : int
        Current tick number (0,1,2,...). Used for delayed departure rule.

    Returns
    -------
    stats : dict
        Currently includes:
          - delivered: number of food deliveries to nest this tick
    """
    W, H = world["W"], world["H"]
    chemical = world["chemical"]
    food = world["food"]
    nest_mask = world["nest_mask"]
    nest_scent = world["nest_scent"]

    delivered = 0

    # --- Update each ant (sequentially, like ask turtles [...]) ---
    for i in range(len(ants["who"])):

        # Delayed departure:
        # Ant with who=i starts moving only when tick >= i.
        if ants["who"][i] >= tick:
            continue

        x, y = ants["x"][i], ants["y"][i]

        # --------------------------------------------------------
        # CASE 1: Searching (not carrying food) -> "look-for-food"
        # --------------------------------------------------------
        if not ants["carrying"][i]:

            # If ant stands on a food patch: pick up 1 unit and turn around.
            if food[y, x] > 0:
                ants["carrying"][i] = True         # switch to returning mode
                food[y, x] -= 1                    # remove 1 unit from patch
                ants["heading"][i] = (ants["heading"][i] + 180) % 360  # reverse direction

            else:
                # Otherwise, maybe follow pheromone, but only in the "chemical window".
                c_here = chemical[y, x]
                if (c_here >= params["chem_follow_min"]) and (c_here < params["chem_follow_max"]):
                    uphill(chemical, ants, i, W, H)

        # --------------------------------------------------------
        # CASE 2: Returning (carrying food) -> "return-to-nest"
        # --------------------------------------------------------
        else:
            # If inside nest: drop food, count delivery, turn around, go searching again
            if nest_mask[y, x]:
                ants["carrying"][i] = False
                delivered += 1
                ants["heading"][i] = (ants["heading"][i] + 180) % 360

            else:
                # Not at nest yet:
                # 1) deposit pheromone to mark this returning path
                chemical[y, x] += params["deposit_amount"]
                # 2) steer uphill on nest_scent field to head home
                uphill(nest_scent, ants, i, W, H)

        # After deterministic bias (uphill/turn), add random wiggle,
        # then move one patch forward.
        wiggle(rng, ants, i, params["wiggle_max"], W, H)
        forward(ants, i, params["step_len"], W, H)

    # --- After all ants move: update chemical field globally ---
    world["chemical"] = diffuse_chemical(world["chemical"], params["diffusion_rate"])
    world["chemical"] *= (100.0 - params["evaporation_rate"]) / 100.0

    return {"delivered": delivered}


# ============================================================
# Rendering (food + pheromone overlays + ants + nest marker)
# ============================================================

def render(world: Dict, ants: Dict, params: Dict, ax: Optional[plt.Axes] = None, title: str = "") -> plt.Axes:
    """
    Render the current world state into a matplotlib Axes.

    We overlay:
    - food (Greens colormap)
    - pheromone chemical (Blues colormap)
    Then draw ants and nest marker on top.

    Note: We use origin="upper" to match array indexing (row 0 at the top).
    """
    if ax is None:
        _, ax = plt.subplots(figsize=(6, 6))

    ax.clear()

    # --- Food layer ---
    ax.imshow(
        world["food"],
        origin="upper",
        interpolation="nearest",
        alpha=0.75,
        cmap="Greens",
        vmin=0.0
    )

    # --- Chemical layer ---
    chem = world["chemical"]
    ax.imshow(
        chem if np.max(chem) > 0.1 else np.zeros_like(chem),
        origin="upper",
        interpolation="nearest",
        alpha=0.55,
        cmap="Blues"
    )

    # --- Ants ---
    # Note: arrays are (x,y) but scatter expects x coords then y coords.
    xs = ants["x"]
    ys = ants["y"]
    colors = np.where(ants["carrying"], "orange", "black")
    ax.scatter(xs, ys, s=6, c=colors)

    # --- Nest marker ---
    ax.scatter(
        [world["max_pxcor"]],
        [world["max_pycor"]],
        s=80,
        marker="X",
        color="red"
    )

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)
    return ax


In [ ]:
rng = np.random.default_rng(0)  # the same seed will produce identical runs!

# Ant parameters (tune these to see different emergent behaviors!)
params = dict(
    diffusion_rate=20.0, 
    evaporation_rate=5.0,
    deposit_amount=60.0,
    wiggle_max=40,
    step_len=1,
    chem_follow_min=0.05,
    chem_follow_max=2.0,
)

# world parameters
max_pxcor = 50
max_pycor = 50

# size of the 3 food sources (radius in patches)
food_radius = 5


world = init_world(rng, max_pxcor=max_pxcor, max_pycor=max_pycor, 
                   food_radius=food_radius)
ants = init_ants(rng, world, population=250)

fig, ax = plt.subplots(figsize=(6, 6))

total_delivered = 0
pheromone_spread = []
food_remaining = []
n_searching = []
n_returning = []

for t in range(800):
    stats = step_world(rng, ants, world, params, tick=t)
    total_delivered += stats["delivered"]

    pheromone_spread.append(np.sum(world["chemical"]))
    food_remaining.append(np.sum(world["food"]))
    n_searching.append(np.sum(~ants["carrying"]))
    n_returning.append(np.sum(ants["carrying"]))
    if t % 1 == 0:
        render(world, ants, params, ax=ax,
            title=f"t={t}  delivered={total_delivered}")
        clear_output(wait=True)
        display(fig)
        plt.pause(0.0011)



In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
ax[0, 0].plot(pheromone_spread)
ax[0, 0].set_title("Total pheromone spread over time")
ax[0, 0].set_xlabel("Time")
ax[0, 0].set_ylabel("Pheromone Level")
ax[0, 1].plot(food_remaining)
ax[0, 1].set_title("Total food remaining over time")
ax[0, 1].set_xlabel("Time")
ax[0, 1].set_ylabel("Food Remaining")
ax[1, 0].plot(n_searching, label="Searching")
ax[1, 0].plot(n_returning, label="Returning")
ax[1, 0].set_title("Number of ants in each state over time")
ax[1, 0].set_xlabel("Time")
ax[1, 0].set_ylabel("Number of Ants")
ax[1, 0].legend()
fig.tight_layout()
fig.show()